# AirBnB Data

In this problem, we will predit the price of an AirBnB listing as a function of its attributes. We get our data from this source: https://www.kaggle.com/c/airbnblala/data#

Our first step will be to import all of the modules we need, and then load the data.

In [ ]:
import Pkg
Pkg.add("CSV")
Pkg.add("Plots")
Pkg.add("DataFrames")
Pkg.add("Statistics")

In [ ]:
using Random
Random.seed!(13)

using CSV
using Plots
using DataFrames
using Statistics
using LinearAlgebra

In [ ]:
df = CSV.read("airbnb.csv")

# let's list all available categories, as well as their datatype using the "eltype" function.
feature_names = names(df)
for i in 1:96
    println(string(i), "\t", string(feature_names[i]), "\t\t\t", string(eltype(df[!, i])))
end

## a) Train / Test Split


Columns 61,62, and 63 are price; let's say that column 61 is the label to predict, and let's hide 62 and 63.

We now will generate a "data" collection, a "target" collection (column 61), and create a train/test split so that we can empirically test for overfitting.

To make the 80 / 20 train test split, we are going to shuffle the data, and then select the first $80\%$ as the train data, with $20\%$ held out for validation.

Use the code below to create a train / test split.

In [ ]:
df = df[.!(ismissing.(df[!, :price])), :]; # let's only consider the examples for which the price is known
df = df[shuffle(1:end), :] # we shuffle the data so that our train/test split will be truly random

train_proportion = 0.8
n = size(df, 1)
println("Size of dataset: ", string(n))

# Put the first ntrain observations in the DataFrame df into the training set, and the rest into the test set
ntrain = convert(Int, round(train_proportion*n))

target = df[:, :price]
data = df[:, filter(col -> (col != :price && col!= :weekly_price && col!= :monthly_price), feature_names)]

#Now you will split the data to create training and test sets. 

# the following variable records the features of examples in the training set
train_x = data[
# the following variable records the features of examples in the test set
test_x = data[
# the following variable records the labels of examples in the training set
train_y = target[
# the following variable records the labels of examples in the test set
test_y = target[

# let's take a look
train_x

## b) Real-Valued Data

Your first task will be to fit a linear model (With bias!) to the simplest type of data available - the real-valued parameters. Once you fit your model to the train data, you will need to print train and test mean squared error, as well as plot the predicted vs the real price.

The numeric data includes:

  - host_listings_count
  - host_total_listings_count
  - accomodates
  - bathrooms
  - bedrooms
  - guests_included		
  - extra_people
  - minimum_nights
  - maximum_nights
  - availability_30
  - availability_60
  - availability_90
  - availability_365
  - number_of_reviews
  - review_scores_rating
  - review_scores_accuracy
  - review_scores_cleanliness
  - review_scores_checkin
  - review_scores_communication
  - review_scores_location
  - review_scores_value

It's also important to also include the following numeric data, which are stored as strings:

  - beds
  - security_deposit
  - cleaning_fee
  
For those, we provide a utility function, which interprets any non-numerical value as a "0", which is an accurate interpretation of "NA" in this context.

The block below is starter code that provides lists of these numeric and numeric-as-string features 
in addition to some data processing routines.

In [ ]:
"This function converts strings to floating point values.
Strings that cannot be represented as a number (like NA) are converted to zeros"
function string_to_float(str)
    try
        parse(Float64, str)
    catch
       0.0
    end
end

labels_real = [
  :host_listings_count,
  :host_total_listings_count,
  :accommodates,
  :bathrooms,
  :bedrooms,
  :guests_included,
  :extra_people,
  :minimum_nights,
  :maximum_nights,
  :availability_30,
  :availability_60,
  :availability_90,
  :availability_365,
  :number_of_reviews,
  :review_scores_rating,
  :review_scores_accuracy,
  :review_scores_cleanliness,
  :review_scores_checkin,
  :review_scores_communication,
  :review_scores_location,
  :review_scores_value
]

labels_string = [
    :beds,
    :security_deposit,
    :cleaning_fee
]

Now you will extract the real valued data first.

In [ ]:
# the following variable should have as many columns as real variables, and as many rows as examples in the training set      
train_vals_real = 
# the following variable should have as many columns as real variables, and as many rows as examples in the test set      
test_vals_real =

Now let's extract the string valued data, convert them to floating point numbers, and convert the result to arrays. 

Remember that $f.( \ldots )$ means "apply f elementwise." It's possible to complete this code with one line per expression below.

In [ ]:
# the following variable should have as many columns as string variables, and as many rows as examples in the training set      
train_vals_from_string = convert(Matrix, string_to_float.(
# the following variable should have as many columns as string variables, and as many rows as examples in the test set      
test_vals_from_string = 
        
@assert(eltype(train_vals_from_string) != String)
@assert(eltype(test_vals_from_string) != String)

To produce our training features, use the real valued features and the string valued features, together with an offset. You can do this in one line using the function `hcat`, which concatenates matrices (with an equal number of rows) horizontally.

Do the same with the test features.

In [ ]:
Xtrain = hcat(
Xtest = hcat(

Now fit the linear model, compute and print the MSE, and plot the predicted versus the real data. Some portions below are left for you to fill in.

We provide some useful helper functions below, though we ask you to complete the MSE function.

Plotting all the test data might significantly slow down this notebook; if that happens, feel free to plot just the first couple hundred points.

In [ ]:
"""This function just computes the mean squared error."""
function MSE(y, pred)
    "Fill this in."
end

"""This completed function plots the main diagonal; 
for a "predicted vs true" plot with perfect predictions,
all data lies on this line"""
function plotDiagonal(xmin, xmax)
    xsamples = [xmin, xmax]
    plot!(xsamples, xsamples, color=:black)
end

"""This completed helper funciton plots x vs, y and labels the axes."""
function plotdata(x,y,xname, yname; margin=.05, plotDiag=true, zeromin=false)
    scatter(x,y, label="data")
    xlabel!(xname)
    ylabel!(yname)
    range_y = maximum(y) - minimum(y)
    range_x = maximum(x) - minimum(x)
    if plotDiag
        plotDiagonal(minimum(x)-margin*range_x, maximum(x)+margin*range_x)
    end
    if zeromin
        ylims!((0.0,maximum(y)+margin*range_y))
        xlims!((0.0,maximum(x)+margin*range_x))
    else
        ylims!((minimum(y)-margin*range_y,maximum(y)+margin*range_y))
        xlims!((minimum(x)-margin*range_x,maximum(x)+margin*range_x))
    end
end

"""This completed function plots the predicted labels vs the actual labels
(We only plots the first 1000 points to avoid slow plots.)"""
function plot_pred_true(test_pred, test_y, max_points = 1000)
    plotdata(test_pred[1:max_points], test_y[1:max_points], "Predicted (\$)", "True (\$)", zeromin=true)
end

In [ ]:
# The weights of your linear equation (compute via least squares)
w = 

#Your prediction vectors
train_pred = 
test_pred = 

#The mean squared error for your predictions.
train_MSE = MSE(
test_MSE = MSE(

In [ ]:
println("Train MSE\t", train_MSE)
println("Test MSE \t", test_MSE)

plot_pred_true(test_pred, test_y)

The predictions trend in the right direction, but there is still plenty of room for improvement.

We are missing some of the most important data: the type of rental. After all, a well-reviewed tent will never cost as much as a poorly-reviewed house.

Unfortunately, this data is stored in categorical columns, including:

  - property_type
  - room_type
  - bed_type
  - cancellation_policy
  - host_response_time
  
We will transform them with a one-hot (boolean) encoding. 

There are also many boolean parameters, which might inform the price:

  - require_guest_profile_picture
  - require_guest_phone_verification
  - instant_bookable
  - is_business_travel_ready
  - has_availability
  - is_location_exact
  - host_identity_verified
  - host_has_profile_pic
  - host_is_superhost
  
One additional category takes set values:

  - host_verifications

Let's handle the boolean values first.

## c) Boolean Data

We've provided a helper function to convert the values stored in the original dataset - "t" and "f" - into Julia "true" and "false." Extract the arrays for these parameters, then concatenate these features with the features from the previous part.

Note that "hcat" will smoothly handle type conversions.

In [ ]:
# Converts from "t" and "f" into true and false
function string_to_bool(str)
    str == "t"
end

bool_labels = [
  :require_guest_profile_picture,
  :require_guest_phone_verification,
  :instant_bookable,
  :is_business_travel_ready,
  :has_availability,
  :is_location_exact,
  :host_identity_verified,
  :host_has_profile_pic,
  :host_is_superhost
]

# the following variable should have as many columns as Boolean variables, and as many rows as examples in the training set      
train_bv = 
# the following variable should have as many columns as Boolean variables, and as many rows as examples in the test set      
test_bv = 

@assert(eltype(train_bv) == Bool)
@assert(eltype(test_bv) == Bool)

# concatenate the real and boolean features to form your X matrix
Xtrain = hcat(
Xtest = hcat(

Now all that's left is to again fit a function and compute the MSE.

In [ ]:
w = 

train_pred = 
test_pred = 

train_MSE = 
test_MSE = 

In [ ]:
println("Train MSE\t", train_MSE)
println("Test MSE \t", test_MSE)

plot_pred_true(test_pred, test_y)

That didn't help much! In fact this is not too surprising. The most informative data is categorical for this dataset: a well-reviewed tent will never cost as much as a poorly reviewed full house.

## d) Categorical Data

Transform the data into one-hot vectors. The categories are easy to extract using the "unique" function.

Here, we ask that you contribute to the helper function, onehot, since this transformation is more challenging. Notice that the categories are passed as an argument; this is important in the case that the test set doesn't have a representation for all of the categories in the training set.

Make sure your function is linear in the number of data points (entries in the column) which it processes. You can use the nested loop structure provided, or clever list comprehensions. It is possible (but not required) to use a "Dataframe" object in a clever way to make the function significantly faster.

If a category appears in the test set but not in the training set, the "one hot" vector should be a vector of zeros for that training set parameter.

We provide hcat syntax here that makes applying the onehot function to every categorical column at once relatively easy.

In [ ]:
cat_labels = [
  :property_type,
  :room_type,
  :bed_type,
  :cancellation_policy,
  :host_response_time
]

#Sets of all categories in a particular column
cats_sets = [unique(train_x[:, label]) for label in cat_labels]

"Computes a onehot vector for every entry in column given a set of categories cats"
function onehot(column, cats=unique(column))
    result = zeros( ,  )
    for 
        for 
            
            
            
        end
    end
    result
end


train_cat_vals = hcat([onehot(train_x[:, cat_labels[i]], cats_sets[i]) for i in 1:size(cat_labels, 1)]...)
test_cat_vals = hcat([onehot(test_x[:, cat_labels[i]], cats_sets[i]) for i in 1:size(cat_labels, 1)]...) 

Now include the categorical features along with the rest real, string, and Boolean features and compute the MSE of the resulting model.

In [ ]:
train_vals = 
test_vals = 

w = 

train_pred = 
test_pred = 

train_MSE = MSE(
test_MSE = MSE(
        
println("Train MSE\t", train_MSE)
println("Test MSE \t", test_MSE)

plot_pred_true(test_pred, test_y)

We're making significant improvements! We have reduced the MSE from before by $10 \%$ using this set of parameters.

## e) Set data

There is another type of data - stored in the host_verifications column - that we haven't used yet. This is set data, best encoded with a many-hot vector.

This particular column does not turn out to be particularly informative, but it is valuable to learn how to work with sets.

Notice is that the set data happens to be stored in a format that is very close to a valid declaration of a Julia string array:

In [ ]:
print(train_x[1, :host_verifications])

All we need to do is run "replace" to fix the single quotes into double quotes:

In [ ]:
replace(train_x[1, :host_verifications], "'" => "\"")

Why is that an improvement? Well, let's talk about how Julia handles input in general. Every input first appears as a string, like this:

In [ ]:
command = "2 + 2"

This command is then parsed into an intermediate representation known as an "Expr" type. This is useful for Julia so that it can break apart the epxression into underlying parts later, and evaluate intermediate results as they are needed (we won't go into much detail about interpreted languages here):

In [ ]:
ex = Meta.parse(command)
typeof(ex)

To go from expression to result, the "eval" command is used:

In [ ]:
eval(ex)

With all this in mind, write a simple one line function which takes one of the strings in the column :host_verifications, and outputs the corresponding String array:

In [ ]:
sample_input = "['email', 'facebook']"
desired_ouput = ["email", "facebook"]

prepro1(s) = 

# The code below checks that prepro1 consumes sample_input to produce desired_output.
# Also check it on the sample below:

a = prepro1(train_x[1, :host_verifications])
println(a)
println(typeof(a))

### Important: Be very careful using this technique in practice! 
Directly parsing and executing data that comes in String form opens a fantastic door to hackers, especially if you are gathering data in an online way for any institution that handles money. If you aren't careful, what I described is a fantastic way to get SQL-injected. In this case, however, we have full knowledge of the data and its source, and it simply makes more sense to use Julia's built-in parsing functions than write our own from scratch (not that this would be a true hurdle). Usually, before handling data in this way, we would need to do substantially more sanitization. It's good to see a little under-the-hood and understand the tools that you are using - similar functions are avaialable in Python and other interpreted languages - but it's also very important to know the vulnerabilities of your code.



We can now apply this function elementwise, to the entire column:

In [ ]:
verif_tr = prepro1.(train_x[:, :host_verifications])
verif_te = prepro1.(test_x[:, :host_verifications])

verif_tr[1]

By the way, if you are working on your project, there is a decent chance that sets were stored in a format similar to what we saw here. Using similar string compositions, you will likely be able to similarly parse the data into Julia container types (whether arrays or proper Julia sets)

Now, take this array of arrays, and turn it into a many-hot vector. You are welcome to use the nested loop structure below, or array comprehensions, or even the DataFrame object if you wish.

In [ ]:
# This is the collection of all types we need to worry about.
# Notice the use of "vcat," which is like "hcat" but vertical
vtypes = unique(vcat(verif_tr...))


"Computes a manyhot vector for every entry in an array of arrays given a set of categories cats"
function manyhot(column, cats=unique(col))
    result = zeros( , )
    for 
        set_of_descriptions = 
        for description in set_of_descriptions
            for 
                
                
                
            end
        end
    end
    result
end


vtr_vals = manyhot(verif_tr, vtypes)
vte_vals = manyhot(verif_te, vtypes)

print(vtypes)
vtr_vals

We will attach it to the full list of values used so far, but it won't change the MSE much, unfortunately:

In [ ]:
train_vals = hcat(
test_vals = hcat(

w = 

train_pred = 
test_pred = 

train_MSE = MSE(
test_MSE = MSE(
        
println("Train MSE\t", train_MSE)
println("Test MSE \t", test_MSE)

plot_pred_true(test_pred, test_y)

## f) Location, location, location

If you remember, a common refrain in Real Estate is "location, location, location." We're treating Manhattan properties with the same formula as Staten Island!

The most naive way to include location data would be with the categories "latitude" and "longitude" - after all, these are continuous variables, so why not include them in the simplest manner possible?

Load the longitude and lattitude data in, then fit a model to predict the labels using *only* these variables.

In [ ]:
loc_labels = [
    :latitude,
    :longitude
]

# Extract these as real values, append a bias (all ones) feature, 
# fit a least squares model, and compute the MSE.

train_MSE = MSE(
test_MSE = MSE(
        
println("Train MSE\t", train_MSE)
println("Test MSE \t", test_MSE)

plot_pred_true(test_pred, test_y)

Clearly, this predictor is insufficient. Price is a nonlinear function of location; simply going north or south does not linearly affect the rental price.

There are several solutions. We will try allowing higher order polynomials in longitude and latitude:

In [ ]:
# create arrays of polynomial features in latitude and longitude up to order 2
train_lv2 = hcat(
    train_loc_vals[:, 1], train_loc_vals[:, 2],
    train_loc_vals[:, 1].^2, train_loc_vals[:, 1].*train_loc_vals[:, 2], train_loc_vals[:, 2].^2,
    ones(size(train_loc_vals, 1))
)
    
test_lv2 = hcat(
    test_loc_vals[:, 1], test_loc_vals[:, 2],
    test_loc_vals[:, 1].^2, test_loc_vals[:, 1].*test_loc_vals[:, 2], test_loc_vals[:, 2].^2,
    ones(size(test_loc_vals, 1))
)

# build a model using these polynomial features of latitude and longitude
w_loc = 

train_pred = 
test_pred = 

train_MSE = MSE(
test_MSE = MSE(
        
println("Train MSE\t", train_MSE)
println("Test MSE \t", test_MSE)

plot_pred_true(test_pred, test_y)

The polynomial expansion is not particularly useful (or sensible) for location. Instead, let's consider a simpler feature transformation. 

Every location can be associated not only with its longitude and latitude, but also with a number of other categorical location parameters. Ones that appear in this dataset include:

  - street
  - neighbourhood
  - neighbourhood_cleansed
  - neighbourhood_group_cleansed
  - city
  - state
  - zipcode
  - market
  - smart_location
  - country_code
  - country
  
Let's see if zipcode predicts price well.

In the next cell, build a model that predicts price using ONLY a one-hot encoding of zipcode. Print the train and test MSE.

Not bad for one (categorical) feature! Now fit a model on both the one-hot zip code together with the other features that we've already accumulated from the previous parts. Print the MSE scores and draw the plots for the full set using the plotting functions provided.

This is by far the best result so far. It seems that location brought in some new information, giving us significantly more predictive power.

Can we do better by using text data?

## g) Text

The following columns consist of long-form textual descriptions:

  - name
  - summary
  - space
  - neighborhood_overview
  - notes
  - transit
  - access
  - interaction
  - house_rules

We will experiment with using features from a pretrained neural network to represent this data.
The neural network has learned an internal representation (say, at some hidden layer of the network) that it uses to make its predictions on a different task (such as sentiment prediction for sentences). 

We will use this internal representation directly for our task of AirBnB price prediction.
Using an internal representation from an unrelated model as a feature is a common technique known as transfer learning.

We will use pretrained features from the Universal Sentence Encoder (USE), which was designed explicitly for transfer learning to different tasks. 
[This paper](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/46808.pdf) documents how this neural network architecture was designed and trained: 
This network consumes text of arbitrary length, and produces a feature vector of length $512$. 

Your TAs have precomputed these features for all of the columns above into the table "airbnb-use-embeddings.csv",
available [as a zip file on the course website](https://people.orie.cornell.edu/mru8/orie4741/homework/airbnb-use-embeddings.zip).
The id of each AirBnB listing is provided, along with features with names of the form "column_number:feature_number". That is, the 468th feature of the 5th text data column (also known as the "name" column) will be found at '5:468'.

You can use [this demo code](https://tfhub.dev/google/universal-sentence-encoder/2)) to explore the USE embedding further. If you would like to use this (or any other) embedding for your project, you are welcome to come to Office Hours and we will gladly help you with your project.

Returning to the AirBnB dataset, we can import the data into a new dataframe. Since our training data has been shuffled, we use a dataset join to assemble the data below.

In [ ]:
df_tf = CSV.read("airbnb-use-embeddings.csv")

train_all = join(train_x, df_tf, on=:id, kind=:left)
test_all = join(test_x, df_tf, on=:id, kind=:left)

In [ ]:
print(size(train_x))
train_embed = convert(Matrix, train_all[:, 97:end])
test_embed = convert(Matrix, test_all[:, 97:end])

To find out how useful these parameters are (or aren't), fit a linear model of price as a function of embedding only (with offset!). Keep in mind that the backslash operator may require several minutes of computation for such a large problem.

Two things are readily apparent: this embedding provides a lot of information, but we are massively overfitting. This is to be expected: after all we have $4689$ parameters to fit, but only $20000$ training points.

Now, is this embedding is capturing new information compared to the other columns? 
To test this, fit a least squares predictor using all of the features we have discussed in this notebook.
Compute MSE and plot the predicted vs expected score.

## h) Further improvements?

In the above, we have seen the strength of adding more (useful) features in improving both training and test performance. In this part, we want to test the idea of whether we can further improve our model by not adding more features, but making wise use of the features.

First, we prepare a data-subsampled dataset with only 20% of the training and test data points retained. This saves time for us to test more settings.

In [ ]:
Random.seed!(0)

# subsample the features and labels of the training and test sets
data_sampling_ratio = 0.2;
train_n_rows_to_sample = convert(Int, round(data_sampling_ratio * size(train_vals)[1]));
train_idx_rows_to_sample = rand(1:size(train_vals)[1], train_n_rows_to_sample);

train_vals_real_small = train_vals_real[train_idx_rows_to_sample, :];
train_vals_from_string_small = train_vals_from_string[train_idx_rows_to_sample, :];
train_bv_small = train_bv[train_idx_rows_to_sample, :];
train_cat_vals_small = train_cat_vals[train_idx_rows_to_sample, :];
train_set_vals_small = train_set_vals[train_idx_rows_to_sample, :];
train_zip_small = train_zip[train_idx_rows_to_sample, :];
train_embed_small = train_embed[train_idx_rows_to_sample, :];

train_vals_small = hcat(
    train_vals_real_small, train_vals_from_string_small, train_bv_small,
    train_cat_vals_small, train_set_vals_small, train_zip_small,
    train_embed_small, ones(size(train_bv_small, 1))
)

test_n_rows_to_sample = convert(Int, round(data_sampling_ratio * size(test_vals)[1]));
test_idx_rows_to_sample = rand(1:size(test_vals)[1], test_n_rows_to_sample);

test_vals_real_small = test_vals_real[test_idx_rows_to_sample, :];
test_vals_from_string_small = test_vals_from_string[test_idx_rows_to_sample, :];
test_bv_small = test_bv[test_idx_rows_to_sample, :];
test_cat_vals_small = test_cat_vals[test_idx_rows_to_sample, :];
test_set_vals_small = test_set_vals[test_idx_rows_to_sample, :];
test_zip_small = test_zip[test_idx_rows_to_sample, :];
test_embed_small = test_embed[test_idx_rows_to_sample, :];

test_vals_small = hcat(
    test_vals_real_small, test_vals_from_string_small, test_bv_small,
    test_cat_vals_small, test_set_vals_small, test_zip_small,
    test_embed_small, ones(size(test_bv_small, 1))
)

# note that we also need to subsample the labels, and use those labels for all model fitting and evaluations on the data-subsampled dataset
train_y_small = train_y[train_idx_rows_to_sample];
test_y_small = test_y[test_idx_rows_to_sample, :];

In the next cell, train the linear regressor on the data-subsampled training set, and take a look at training and test MSEs.

In [ ]:
w = 

# "small" means the data points are subsampled
train_pred_small = 
test_pred_small = 

train_MSE_small = MSE(
test_MSE_small = MSE(

println("Train MSE\t", string(train_MSE_small))
println("Test MSE \t", string(test_MSE_small))

plot_pred_true(test_pred_small, test_y_small)

**Question h1**: Compare the training and test MSEs on the data-subsampled dataset with those on the original dataset: the one that includes all data points and all features, as you did in the cell above the heading of this section. For these two fitted models, are they overfitting or underfitting? Which one over/underfits more? Why?

Answer to Question h1:

What we are going to try first is to randomly select only a subset of the **text embedding** features for model fitting and prediction. In the next cell, randomly sample 10% of the text embedding features **from the data-subsampled training and test datasets** (so that the following steps would not take too much time) and **retain all the other features (including the offest)**. We call this the entire data-subsampled dataset (i.e., not feature-subsampled) in the following context. Then, use the new set of features to fit a linear regression model on the training set and predict on the test set. Print the training and test MSEs.

In [ ]:
Random.seed!(0)

# "sub" means feature subsampling
sampling_ratio = 0.1;

train_embed_sub = 
test_embed_sub = 

train_vals_sub = hcat(
    train_vals_real_small, train_vals_from_string_small, train_bv_small,
    train_cat_vals_small, train_set_vals_small, train_zip_small,
    train_embed_sub, ones(size(train_bv_small, 1))
)
test_vals_sub = hcat(
    test_vals_real_small, test_vals_from_string_small, test_bv_small,
    test_cat_vals_small, test_set_vals_small, test_zip_small,
    test_embed_sub, ones(size(test_bv_small, 1))
)

w_sub = 

train_pred_sub = 
test_pred_sub = 

train_MSE_sub = MSE(
test_MSE_sub = MSE(

println("Train MSE\t", string(train_MSE_sub))
println("Test MSE \t", string(test_MSE_sub))

plot_pred_true(test_pred_sub, test_y_small)

In the next cell, test on a collection of the number of text embedding features to sample, and plot the change of train and test MSEs. This may take several minutes, but you would not need to do it multiple times as long as your code in the above cell works well. 

In [ ]:
train_MSE_sub_all = []
test_MSE_sub_all = []

# the collection of the number of text embedding features to sample
n_embed_to_sample_all = [10, 50, 100, 200, 500, 1000, 2000, 3000, 4000]

for n_embed_to_sample in n_embed_to_sample_all
    println("number of text embedding features to sample: ", string(n_embed_to_sample))    
    Random.seed!(n_embed_to_sample)

    train_embed_sub = 
    test_embed_sub = 

    train_vals_sub = hcat(
        train_vals_real_small, train_vals_from_string_small, train_bv_small,
        train_cat_vals_small, train_set_vals_small, train_zip_small,
        train_embed_sub, ones(size(train_bv_small, 1))
    )
    test_vals_sub = hcat(
        test_vals_real_small, test_vals_from_string_small, test_bv_small,
        test_cat_vals_small, test_set_vals_small, test_zip_small,
        test_embed_sub, ones(size(test_bv_small, 1))
    )

    w_sub = 

    train_pred_sub = 
    test_pred_sub = 

    train_MSE_sub = 
    test_MSE_sub = 

    println("Train MSE\t", string(train_MSE_sub))
    println("Test MSE \t", string(test_MSE_sub))
    
    append!(train_MSE_sub_all, train_MSE_sub)
    append!(test_MSE_sub_all, test_MSE_sub)  

end

In the next cell, plot the change of training and test MSEs with the number of sampled features, together with two horizontal lines showing the train and test MSEs on the entire data-subsampled dataset (not feature-subsampled).

**Question h2**: 
- In the plot you get above:does the model over/underfits worse when the number of sampled text embedding feature increases? Why?
- Compared to the model performance on the entire data-subsampled dataset (i.e., not feature-subsampled), how does feature subsampling works, in terms of prediction error and generalization?

Answer to Question h2:

Finally, we resort to a "smarter" way of reducing the number of features. We resort to Principal Component Analysis (PCA) on the embedding features, which seeks for its low dimensional representation. 

In the next cell, write the (standard) code for fitting and predicting the linear regression model on the dimensionality-reduced training and test sets. Be aware that this cell may take several minutes to run. 

In [ ]:
train_MSE_pca_all = []
test_MSE_pca_all = []

println("Train MSE on the data-subsampled dataset \t", string(train_MSE_small))
println("Test MSE on the data-subsampled dataset \t", string(test_MSE_small))

# do PCA on the training features
U, S, V = svd(train_embed_small);

k_all = [10, 50, 100, 500, 1000]


for k in k_all
    println(k)
    
    Uk = U[:, 1:k];
    Sk = S[1:k];
    Vk = V[:, 1:k];
    
    train_embed_pca =  train_embed_small * Vk
    test_embed_pca = test_embed_small * Vk
    
    # training set
    train_vals_pca = hcat(
    train_vals_real_small, train_vals_from_string_small, train_bv_small,
    train_cat_vals_small, train_set_vals_small, train_zip_small,
    train_embed_pca, ones(size(train_bv_small, 1))
    )
    
    # test set
    test_vals_pca = hcat(
        test_vals_real_small, test_vals_from_string_small, test_bv_small,
        test_cat_vals_small, test_set_vals_small, test_zip_small,
        test_embed_pca, ones(size(test_bv_small, 1))
    )
    
    # write your code for model fitting and prediction
    w_pca = 

    train_pred_pca = 
    test_pred_pca = 

    train_MSE_pca = MSE(
    test_MSE_pca = MSE(

    println("Train MSE\t", string(train_MSE_pca))
    println("Test MSE \t", string(test_MSE_pca))
    
    append!(train_MSE_pca_all, train_MSE_pca)
    append!(test_MSE_pca_all, test_MSE_pca)    
    
end

In the next cell, plot the change of train and test MSEs in the PCA case, together with two horizontal lines showing the train and test MSEs on the entire data-subsampled dataset (i.e., not feature-subsampled). This is similar to what you did in text embedding feature subsampling.

**Question h3**: 
- In the PCA plot you get above, does the model over/underfits worse when the number of sampled text embedding feature increases? 
- Compared to the model performance on **both** the entire data-subsampled dataset (i.e., not feature-subsampled) and the PCA-dimensionality-reduced dataset, how does PCA works, in terms of prediction error and generalization?

Answer to Question h3: